# Задание 1

[Свертка списка](https://ru.wikipedia.org/wiki/%D0%A1%D0%B2%D1%91%D1%80%D1%82%D0%BA%D0%B0_%D1%81%D0%BF%D0%B8%D1%81%D0%BA%D0%B0) -  это обобщенная операция над списком, c помощью которой можно преобразовать список в единое значение. Например, рассмотрим реализации свертки слева и свертки справа (левоассоциативную свертку и правоассоциативную свертку):

In [1]:
def foldl(f, x0, lst):
    if not lst:
        return x0
    return foldl(f, f(x0, lst[0]), lst[1:])

def foldr(f, x0, lst):
    if not lst:
        return x0
    return f(lst[0], foldr(f, x0, lst[1:]))

In [2]:
#сумма
foldl(lambda x, y: x + y, 0, [1, 2, 3])

6

In [3]:
#конкатенация
foldl(lambda x, y: '{}{}'.format(x, y), '', [1, 2, 3])

'123'

In [4]:
f = lambda x, y: x / y
foldl(f, 1, [1, 2, 3]), \
foldr(f, 1, [1, 2, 3])

(0.16666666666666666, 1.5)

Задача: реализовать foldl через foldr и наоборот. Вместо многоточий нужно вставить выражения, которые бы привели к нужному результату. Модифицировать сам список нельзя. 

In [5]:
def foldl2(f, x0, lst):
    return foldr(lambda *x: f(*x[::-1]), x0, lst[::-1])

In [6]:
def foldr2(f, x0, lst):
    return foldl(lambda *x: f(*x[::-1]), x0, lst[::-1])

# Задание 2

нужно написать функцию, которая принимает две строки и проверяет, входит ли хотя бы одна перестановка второй строки в первую. Например:

> a = 'abcrotm'
> 
> b = 'tro'

функция def check_inv(a, b) вернет True, так как 'rot' содержится в 'abcrotm'. Нужно подумать как можно более оптимальный алгоритм и оценить его сложность. 

In [7]:
from collections import Counter


def check_inv(a, b):
    if len(b) > len(a):
        return False

    return Counter(b) - Counter(a) & Counter(b) == {}


check_inv('abcrotm', 'tro')

True

# Задание 3

Реализовать бинарное дерево (класс Tree), в нём методы __repr__, __str__, __iter__ (итерация только по листьям).

In [8]:
from collections import deque
import numpy as np

class Tree:
    def __init__(self, value=None, left=None, right=None):
        self.left = left
        self.right = right
        self.value = value

    def __repr__(self):
        subtree_repr = lambda subtree: f', {subtree.__repr__()}' if subtree is not None else ''

        left_repr = subtree_repr(self.left)
        right_repr = subtree_repr(self.right)
        return f'{type(self).__name__}({self.value}{left_repr}{right_repr})'

    def get_levels(self):
        '''BFS для получения списков вершин по уровням дерева'''
        q = deque()
        if self is not None:
            q.append(self)
        levels = []
        next_level = []
        while q:
            level = next_level
            next_level = []
            for i in range(len(q)):
                node = q.popleft()
                if node is None:
                    level.append(None)
                    next_level.append(None)
                    next_level.append(None)
                else:
                    q.append(node.left)
                    q.append(node.right)
                    level.append(node.value)
            levels.append(level)
        return levels[:-1]

    def __str__(self):
        s = ''
        levels = self.get_levels()
        gap = 2 ** (len(levels) - 1) - 1
        for i, level in enumerate(levels):
            for j, val in enumerate(level):
                s += f'{val}' if val is not None else ' '
                s += ' ' * gap
            gap = 2 ** (len(levels) - 1 - i) - 1
            s += '\n'
        return s


    def __iter__(self):
        if self.left is None and self.right is None:
            yield self.value

        if self.left is not None:
            for tree in self.left:
                yield tree

        if self.right is not None:
            for tree in self.right:
                yield tree


tree = Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))

print(tree)
print(tree.__repr__())
list(tree) == [3, 4, 2]

0   
1   2   
3 4     

Tree(0, Tree(1, Tree(3), Tree(4)), Tree(2))


True

# Задание 4

Реализовать простейший калькулятор математических выражений:
- только целые числа
- **+**, **\-**, **\***, **\**
- скобки

**Можно использовать регулярные выражения**

In [9]:
import re


def compare_operators(stack, x):
    precedence = {'+': 1, '-': 1, '*': 2, '/': 2}
    try:
        return precedence[x] <= precedence[stack[-1]]
    except KeyError:
        return False


def infix_to_postfix(inputs):
    outputs = []
    stack = []
    for x in inputs:
        if x.isdigit():
            outputs.append(int(x))
        elif x == '(':
            stack.append(x)
        elif x == ')':
            while stack and stack[-1] != '(':
                outputs.append(stack.pop())
            stack.pop()
        else:
            while stack and compare_operators(stack, x):
                outputs.append(stack.pop())
            stack.append(x)
    while stack:
        outputs.append(stack.pop())
    return outputs


def calc(expr):
    infix = re.findall(r'[-+/*()]|\d+(?:\.\d+)?', expr)
    postfix = infix_to_postfix(infix)

    stack = []
    for x in postfix:
        if type(x) is int:
            stack.append(x)
        else:
            operand2, operand1 = stack.pop(), stack.pop()

            if x == '+':
                stack.append(operand1 + operand2)
            elif x == '-':
                stack.append(operand1 - operand2)
            elif x == '*':
                stack.append(operand1 * operand2)
            elif x == '/':
                stack.append(operand1 // operand2)

    return stack.pop()


calc('2 * (15 - 3 * 4) - 2') == 4

True